<a href="https://colab.research.google.com/github/hansong0219/Advanced-DeepLearning-Study/blob/master/improved_GAN/ACGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ACGAN 

**ACGAN(보조 분류 GAN)** 은 원리상 Conditional GAN 과 유사한 형태를 가진다. CGAN 의 경우, 판별기는 입력으로 이미지와 그 레이블을 받고 그 이미지가 진짜일 확률을 출력한다. 그러나 ACGAN 에서의 입력은 이미지이고 출력은 이미지가 진짜인지의 확률과 어떤 클래스인지를 동시에 출력하게 된다. 

기본적으로 CGAN 에서는 부가정보를 네트워크에 제공하지만, ACGAN에서는 이를 제공하지 않고 보조 클래스 디코더 네트워크(Axuiliary class decoder)를 이용하여 부가 정보를 재구성한다.

ACGAN 에는 추가적으로 분류기의 손실함수가 주어지는데 진짜와 가짜 이미지를 추가적으로 분류하는 작업을 수행한다. 
또, 생성기의 손실함수에서도 판별기를 속이는 일 외에도 가짜이미지를 정확하게 분류했는지도 손실을 계산하게된다.

#GPU 할당

Colab 이 아닌 환경에서 아래의 코드를 통해 우선적으로 gpu를 할당해준다.

In [ ]:
import tensorflow as tf 
physical_devices =tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0],True)

In [1]:
from tensorflow.keras.layers import Activation, Dense, Input
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.layers import Reshape, Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K

import numpy as np
import math
import matplotlib.pyplot as plt

# 생성기 및 판별기 함수구성
CGAN 의 코드를 기반으로 하여 구성한다. 

In [2]:
def build_discriminator(inputs, activation='sigmoid', num_labels=None, num_codes=None):
  """
  판별기 모델구성 
  가짜와 진짜를 판별하기 위한 LeakyReLU - Conv2D 로 구성된 스택이다.
  이 네트워크는 BN 으로 수렴되지 않으므로, 사용하지 않는다.
  """

  # 입력 인수
  """
  inputs(layer) : 판별기의 입력 계층(이미지)
  activation (string) : 출력 활성화 계층의 이름
  num_labels (int): ACGAN 과 InfoGAN 에서의 원-핫 레이블의 차원
  num_codes(int) : Q network as output
  """

  #출력 인수 : Model: 판별기 모델

  kernel_size = 5
  layer_filters = [32, 64, 128, 256]

  x = inputs
  for filters in layer_filters:
    # 첫 세 개의 합성곱 계층에서는 strides = 2 를 사용
    # 마지막 한 계층은 strides = 1을 사용
    
    if filters == layer_filters[-1]:
      strides = 1
    else:
      strides = 2

    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(filters = filters, kernel_size = kernel_size, strides=strides, padding='same')(x)

  x = Flatten()(x)
  outputs = Dense(1)(x)
  if activation is not None:
    outputs = Activation(activation)(outputs)

  if num_labels:
    # ACGAN 과 InfoGAN 에는 두번째 출력이 있음
    # MNIST 에서 두 번째 출력은 10차원 원-핫 레이블 벡터로 구성됨

    layer = Dense(layer_filters[-2])(x)
    labels = Dense(num_labels)(layer)
    labels - Activation('softmax',name='label')(labels)
    if num_codes is None:
      outputs = [outputs, labels]

    else:
      #InfoGAN 에는 세 번쨰와 네 번째 출력이 있음
      #세 번째 출력: x 가 주어졌을 때 첫번째 c 를 나타내는 1차원 연속 Q

      code1 = Dense(1)(layer)
      code1 = Activation('sigmoid', name='code1')(code1)

      #네 번째 출력: x가 주어졌을 때 두 번째 c를 나타내는 1차원 연속 Q
      code2 = Dense(1)(layer)
      code2 = Activation('sigmoid',name='code2')(code2)

      outputs = [outputs, labels, code1, code2]

  elif num_codes is not None:
    #z0_recon: z0 정규 분포를 재구성
    z0_recon = Dense(num_codes)(x)
    z0_recon = Activation('tanh', name='z0')(z0_recon)

    outputs = [outputs, z0_recon]

  return Model(inputs, outputs, name='discriminator')

In [12]:
def build_generator(inputs, image_size, activation='sigmoid', labels=None, codes=None):
  """
  생성기 모델 구성
  가짜 이미지를 생성하는 BN-ReLU-Conv2DTranspose 스택으로 이루어진다.
  출력 활성화로 sigmoid 를 사용한다.
  """
  
  #입력 인수
  """
  inputs (Layer): 생성기의 입력계층 (z-vector)
  image_size (int) : 한 변의 목표크기 (정사각형 이미지를 가정)
  activation(string) : 출력 활성화 계층 이름
  labels (tensor) : 입력 레이블
  codes (list) : InfoGAN 에서 사용하는 2차원 풀링 코드
  """

  # 출력 인수: Model : 생성기 모델

  image_resize = image_size//4
  
  # 네트워크 매개 변수
  kernel_size = 5
  layer_filters = [128, 64, 32, 1]
  
  if labels is not None:
    if codes is None:
      #ACGAN 레이블
      # z 노이즈 벡터와 원-핫 레이블을 연결
      inputs = [inputs, labels]

    else:
      #InfoGAN 코드
      #z 노이즈 벡터와 원-핫 레이블과 코드 1, 2 를 연결
      inputs = [inputs, labels] + codes
    
    x = concatenate(inputs,axis=1)
      
  elif codes is not None:
    #StackedGAN 의 생성기
    inputs = [inputs, codes]
    x = concatenate(inputs,axis=1)
  
  else:
    x = inputs

  x = Dense(image_resize*image_resize*layer_filters[0])(x)
  x = Reshape((image_resize,image_resize,layer_filters[0]))(x)

  for filters in layer_filters:
    # 처음 두 합성곱 계층은 strides = 2 를 사용하여 이미지 사이즈를 변경함
    # 마지막 두계층은 strides = 1을 사용
    if filters > layer_filters[-2]:
      strides = 2
    else:
      strides = 1
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(filters = filters, kernel_size = kernel_size, strides = strides, padding='same')(x)

  if activation is not None:
    x = Activation(activation)(x)

  return Model(inputs, x, name='generator')

# 유틸 함수 정의

In [3]:
def plot_images(generator,
                noise_input,
                noise_class,
                show=False,
                step=0,
                model_name="gan"):

    os.makedirs(model_name, exist_ok=True)
    filename = os.path.join(model_name, "%05d.png" % step)
    images = generator.predict([noise_input, noise_class])
    print(model_name , " labels for generated images: ", np.argmax(noise_class, axis=1))
    plt.figure(figsize=(2.2, 2.2))
    num_images = images.shape[0]
    image_size = images.shape[1]
    rows = int(math.sqrt(noise_input.shape[0]))
    for i in range(num_images):
        plt.subplot(rows, rows, i + 1)
        image = np.reshape(images[i], [image_size, image_size])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.savefig(filename)
    if show:
        plt.show()
    else:
        plt.close('all')

# 데이터 세트 로딩

In [4]:
# MNIST 불러오기 
(x_train, y_train), (_, _) = mnist.load_data()

In [5]:
# CNN 을 위한 데이터형상 조정 및 정규화
image_size = x_train.shape[1]
x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
x_train = x_train.astype('float32')/255

# 라벨 지정
num_labels = np.amax(y_train) + 1
y_train = to_categorical(y_train)

# ACGAN 모델 구성

In [6]:
model_name = "acgan_mnist"

# 네트워크 매개 변수 지정
latent_size = 100
batch_size = 64
train_steps = 40000
lr = 2e-4
decay = 6e-8
input_shape = (image_size, image_size, 1)
label_shape = (num_labels, )

In [7]:
# 판별기 모델 구성
inputs = Input(shape = input_shape, name = 'discriminator_input')

# 예측 소스와 레이블, 두개의 출력으로 판별기 빌더 함수 호출
discriminator = build_discriminator(inputs, num_labels=num_labels)

# RMSProp 을 사용할 때 판별기가 쉽게 수렴한다.
optimizer = RMSprop(lr=lr, decay=decay)

# 손실함수는 두개를 사용한다 - 1) 이미지가 진짜일 확률, 2) 이미지의 클래스 레이블이 맞을 확률
loss = ['binary_crossentropy', 'categorical_crossentropy']

discriminator.compile(loss=loss, optimizer=optimizer, metrics = ['accuracy'])

discriminator.summary()

Model: "discriminator"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
discriminator_input (InputLayer [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 28, 28, 1)    0           discriminator_input[0][0]        
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 14, 14, 32)   832         leaky_re_lu[0][0]                
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 14, 14, 32)   0           conv2d[0][0]                     
______________________________________________________________________________________

In [13]:
# 생성기 모델 구성
input_shape = (latent_size,)
inputs = Input(shape=input_shape, name='z_input')
labels = Input(shape=label_shape, name='labels')

#입력 레이블로 생성기 빌더 함수 호출
generator = build_generator(inputs, image_size, labels=labels)
generator.summary()

Model: "generator"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
z_input (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
labels (InputLayer)             [(None, 10)]         0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 110)          0           z_input[0][0]                    
                                                                 labels[0][0]                     
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 6272)         696192      concatenate_1[0][0]      

In [14]:
# 적대적 모델 구성 
optimizer = RMSprop(lr=lr*0.5, decay=decay*0.5)
discriminator.trainable = False
adversarial = Model([inputs,labels], discriminator(generator([inputs, labels])), name=model_name)

#판별기와 동일한 2개 손실 함수 사용: 1) 이미지가 진짜일 확률, 2) 이미지 클래스 레이블이 맞을 확률
adversarial.compile(loss=loss, optimizer=optimizer, metrics = ['accuracy'])
adversarial.summary()

Model: "acgan_mnist"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
z_input (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
labels (InputLayer)             [(None, 10)]         0                                            
__________________________________________________________________________________________________
generator (Functional)          (None, 28, 28, 1)    1364225     z_input[0][0]                    
                                                                 labels[0][0]                     
__________________________________________________________________________________________________
discriminator (Functional)      [(None, 1), (None, 1 1606283     generator[0][0]        

In [15]:
#훈련함수 input 형태로 변경
models = (generator, discriminator, adversarial)
data = (x_train, y_train)
params = (batch_size, latent_size, train_steps, num_labels, model_name)

In [16]:
def train(models, data, params):
  """
  판별기와 적대적 네트워크를 배치단위로 교대로 훈련 
  먼저 판별기가 진짜와 가짜 이미지, 그에 해당하는 원- 핫 레이블을 사용해 훈련됨
  다음으로 적대적 네트워크가 진짜인 척하는 가짜 이미지와 그에 해당하는 원-핫 레이블 사용해 운련됨
  """

  # 입력 인수 
  """
  models (list): 생성기, 판별기, 적대적 모델
  data(list): x_train, y_train 데이터
  params(list): 네트워크 매개변수
  """

  # GAN 모델 
  generator, discriminator, adversarial = models
  # 이미지 레이블 
  x_train, y_train = data  
  # 네트워크 매개변수
  batch_size, latent_size, train_steps, num_labels, model_name = params

  # 생성기의 이미지는 500 epochs 마다 저장됨
  save_interval = 500
  # 훈련하는 동안 생성기 출력을 보여주기 위한 노이즈 벡터
  noise_input = np.random.uniform(-1.0, 1.0, size=[16,latent_size])

  #노이즈에 조건을 부여할 원-핫 레이블
  noise_class = np.eye(num_labels)[np.arange(0, 16) % num_labels]
  
  # 훈련 데이터 세트의 요소 개수 
  train_size = x_train.shape[0]

  print(model_name, "Labels for generated images:", np.argmax(noise_class,axis=1))

  for i in range(train_steps):
    # 1 배치 에 대한 판별기 훈련 
    
    # 데이터 세트에서 진짜 이미지를 임의로 선정
    rand_indices = np.random.randint(0, train_size, size=batch_size)
    real_images = x_train[rand_indices]
    # 대응하는 원-핫 레이블 생성 
    real_labels = y_train[rand_indices]

    #생성기를 사용해 노이즈로 부터 가짜 이미지 생성
    noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
    
    #임의의 원-핫 레이블 할당
    fake_labels = np.eye(num_labels)[np.random.choice(num_labels, batch_size)]
    #가짜레이블 조건으로 하는 가짜이미지 생성
    fake_images = generator.predict([noise, fake_labels])

    #진짜 이미지 + 가짜이미지 를 1배치 훈련데이터로 사용
    x = np.concatenate((real_images, fake_images))
    labels = np.concatenate((real_labels, fake_labels))

    #진짜와 가짜 이미지에 레이블을 붙임
    #진짜이미지 레이블은 1, 가짜 이미지 레이블은 0
    y = np.ones([2*batch_size,1])
    y[batch_size:,:] = 0.0

    #판별기 네트워크 훈련, 손실과 정확도 기록
    #['loss', 'activation_1_loss', 'label_loss, 'activation_1_acc', ' label_acc'] 로 생성
    metrics = discriminator.train_on_batch(x,[y, labels])
    fmt = "%d: [disc loss: %f, srcloss: %f, lblloss:$f, srcacc: %f, lblacc: %f]"
    log = fmt %(i, metrics[0],metrics[1],metrics[2],metrics[3],metrics[4])

    #1 배치에 대한 적대적 네트워크 훈련
    #레이블이 1인 가짜 원핫 레이블을 조건으로 하는 가짜 이미지의 1배치
    #적대적 네으퉈크에서 파별기의 가중치는 고정되기 때문에 생성기만 훈련이된다. 

    #균등분포를 통해 노이즈 생성 
    noise = np.random.uniform(-1.0, 1.0, size=[batch_size,latent_size])

    #임의의 원-핫 레이블 할당
    fake_labels = np.eye(num_labels)[np.random.choice(num_labels,batch_size)]

    # 가짜 이미지에 진짜로 속이는 레이블을 붙임
    y = np.ones([batch_size, 1])
    
    #적대적 네으퉈크 훈련
    #판별기 훈련과 달리 변수에 가짜 이미지를 별도의 변수로 저장하지 않으며, 분류를 위해 가짜 이미지는 적대적 네트워크의 판별기 입력으로 전달됨

    #log 기록
    metrics = adversarial.train_on_batch([noise, fake_labels],[y, fake_labels])
    fmt = "%s: [advr loss: %f, srcloss: %f, lblloss:$f, srcacc: %f, lblacc: %f]"
    log = fmt%(log, metrics[0],metrics[1],metrics[2],metrics[3],metrics[4])
    print(log)

    if (i+1) % save_interval == 0:
      if (i+1) == train_steps:
        show = True
      else:
        show = False

      plot_images(generator, noise_input=noise_input,noise_class=noise_class,show=show, step=(i+1), model_name=model_name) 

  generator.save(model_name+".h5")

In [ ]:
train(models, data, params)

In [ ]:
def test_generator(generator, class_label=None):
    noise_input = np.random.uniform(-1.0, 1.0, size=[16, 100])
    step = 0
    if class_label is None:
        num_labels = 10
        noise_class = np.eye(num_labels)[np.random.choice(num_labels, 16)]
    else:
        noise_class = np.zeros((16, 10))
        noise_class[:,class_label] = 1
        step = class_label

    plot_images(generator,
                noise_input=noise_input,
                noise_class=noise_class,
                show=True,
                step=step,
                model_name="test_outputs")

In [ ]:
class_label = None
test_generator(generator, class_label)